Bismillahirrahmanirrahiiiim semoga bisa lancar Aminnnn with Alloh everything is easy, alright, and best also incredibleee

# Image Captioning

## Import Library

In [ ]:
pip install salesforce-lavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import Image
import requests
import torch
# from PIL import Image
from lavis.models import load_model_and_preprocess
import pandas as pd
# For download files in colab local directory
from google.colab import files
from deep_translator import GoogleTranslator
from google.colab import drive

## Permission to Mydrive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Define Path Data

In [ ]:
parent_path = "/content/drive/My Drive/Bismillah_Skripsi_Bisa_Amiiin/Final_Data/"
path_v1 = parent_path + "v1_with_stemming/"
path_v2 = parent_path + "v2_without_stemming/"
path_v1_revisi = parent_path + "v1_version_with_stemming_revisi/"
path_v2_revisi = parent_path + "v2_version_without_stemming_revisi/"
path_v2_revisi2 = parent_path + "v2_version_without_stemming_revisi2/"
path_v3 = parent_path + "v3_version_with_stemming_without_stopword/"
path_v3_revisi = parent_path + "v3_version_with_stemming_without_stopword_revisi/"
path_v4 = parent_path + "v4_version_without_stemming_without_stopword/"
path_v4_revisi = parent_path + "v4_version_without_stopwords_without_stemming/"

## Load Data

In [ ]:
path_data = path_v3_revisi

# train_data = path_data + "train.xlsx"
# dev_data = path_data + "validation.xlsx"
# test_data = path_data + "test.xlsx"

# sub-path train data
path_data_sub_1 = path_data + "train_1.xlsx"
path_data_sub_2 = path_data + "train_2.xlsx"
path_data_sub_3 = path_data + "train_3.xlsx"
path_data_sub_4 = path_data + "train_4.xlsx"

In [ ]:
data = pd.read_excel(path_data_sub_4)
name_file = 'captions_' + 'train_data1' + ".json"
data.head()

,Unnamed: 0.1,Unnamed: 0,name,review_img_url,review_rating,review_text,review_date
0,0,1650,Rumah Makan Layar Seafood Manyar,https://lh5.googleusercontent.com/p/AF1QipPbZl...,1,ramai banget restoran kalau tempat indah pakai...,2022-06-09
1,1,1651,Rumah Makan Layar Seafood Manyar,https://lh5.googleusercontent.com/p/AF1QipN-b7...,1,biasa saja sih untuk makan standard layak rest...,2022-10-24
2,2,1652,Rumah Makan Layar Seafood Manyar,https://lh5.googleusercontent.com/p/AF1QipO3Q9...,1,makan tetap enak namun untuk parkir kalau suda...,2021-10-18
3,3,1653,RM NELAYAN SEAFOOD BELITONG.,https://lh5.googleusercontent.com/p/AF1QipM6s5...,1,saya pesan kepiting bumbu asli dan tomyam untu...,2018-09-01
4,4,1654,RM Sari Laut,https://lh5.googleusercontent.com/p/AF1QipMgX-...,1,menu cukup agam rasa lumayan enak harga saing ...,2018-12-29


In [ ]:
#drop column with index 0 by index
data.drop(data.columns[[0]], axis=1, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      510 non-null    int64 
 1   name            510 non-null    object
 2   review_img_url  510 non-null    object
 3   review_rating   510 non-null    int64 
 4   review_text     510 non-null    object
 5   review_date     510 non-null    object
dtypes: int64(2), object(4)
memory usage: 24.0+ KB


## Fetch Url Data

In [ ]:
df = data[['review_img_url']].astype(str)

In [ ]:
df.head()

,review_img_url
0,https://lh5.googleusercontent.com/p/AF1QipPbZl...
1,https://lh5.googleusercontent.com/p/AF1QipN-b7...
2,https://lh5.googleusercontent.com/p/AF1QipO3Q9...
3,https://lh5.googleusercontent.com/p/AF1QipM6s5...
4,https://lh5.googleusercontent.com/p/AF1QipMgX-...


## Image Captioning Process

### Set the device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

### Initialize Image Captioning Model

In [ ]:
# loads BLIP caption base model, with finetuned checkpoints on MSCOCO captioning dataset.
# this also loads the associated image processors
model, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True, device=device)

100%|██████████| 2.50G/2.50G [00:26<00:00, 102MB/s]


### Image Captioning Implementation

In [ ]:
def image_captioning(url):
  # x = type(url)
  response = requests.get(url, stream=True)
  image = Image.open(response.raw)
  image = image.convert("RGB")
  # preprocess the image
  # vis_processors stores image transforms for "train" and "eval" (validation / testing / inference)
  image = vis_processors["eval"](image).unsqueeze(0).to(device)
  # generate caption
  result = model.generate({"image": image})
  result = ''.join(map(str, result))
  result = GoogleTranslator(source='auto', target='id').translate(result)
  return result

In [ ]:
df['review_img_captioning'] = df['review_img_url'].apply(lambda url: image_captioning(url))

/usr/local/lib/python3.9/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


### Export Data

In [ ]:
pasangan_key_value = df.set_index('review_img_url')['review_img_captioning'].to_dict()

In [ ]:
pasangan_key_value

{'https://lh5.googleusercontent.com/p/AF1QipPbZlr7Zasc6KEZiE7MiSqdQax6BIXEew1_C3pe': 'sebuah ruangan dengan lantai bata dan pohon natal yang menyala',
 'https://lh5.googleusercontent.com/p/AF1QipN-b758Pb2GxjPsNOLDm2NSmw7RbaLlqXw7Fzy1': 'sekelompok orang duduk mengelilingi meja makan makanan',
 'https://lh5.googleusercontent.com/p/AF1QipO3Q9ekbO6RurDv2FtyBf96z6F9gM3gnutL6KkT': 'dua wanita duduk di meja dengan piring makanan',
 'https://lh5.googleusercontent.com/p/AF1QipM6s59J4xy5Y7k96HHH5FvtchaXOxC95ay_Yyy7': 'semangkuk sup dengan sendok di dalamnya',
 'https://lh5.googleusercontent.com/p/AF1QipMgX-3qZdb7jiBx1FrkagUZKdaj3dEvzeRqtQND': 'sebuah restoran dengan bangku dan tanda di dinding',
 'https://lh5.googleusercontent.com/p/AF1QipMGiJGTPKu1oQiVMxSi7nwZlEw0jfuxdSpOsS3u': 'bagian dalam restoran dengan meja dan kursi',
 'https://lh5.googleusercontent.com/p/AF1QipPQSwr_Yh4nUxsVajuGBb3bBcaCwPf0GJXXF8bF': 'sepiring makanan di atas meja dengan garpu dan pisau',
 'https://lh5.googleusercontent

In [ ]:
import json
with open(name_file, 'w+') as f:
  json.dump(pasangan_key_value, f)
  files.download(name_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>